University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2022  
Luigi Acerbi  
Based on notebook by Antti Honkela

# Lecture 1: Floating point numbers and numerics of probabilities

## Floating point number basics

Real numbers are typically represented as floating point numbers in computers. Floating point numbers use a fixed storage size and hence can offer only finite precision. Floating point numbers do not fulfill the usual axioms of real numbers, which means they can sometimes behave in unexpected ways.

Background reading on floating point numbers:

http://floating-point-gui.de/formats/fp/  
http://floating-point-gui.de/errors/rounding/  
http://floating-point-gui.de/errors/comparison/  
http://floating-point-gui.de/errors/propagation/  
https://hal.archives-ouvertes.fr/hal-00128124v5/document  
and references therein.

## Useful links

https://courses.helsinki.fi/fi/aycsm90004en/135221588 : "Data Analysis with Python" MOOC  
http://www.learnpython.org/ : Nice interactive Python tutorial  
https://docs.python.org/3/tutorial/index.html : Official documentation for Python3  
https://docs.scipy.org/doc/numpy/user/quickstart.html : Tutorial for one of the most important Python modules, SciPy  

### 1. Computing with floating point numbers

Write a program to increment `x = 0.0` by `0.1` 100 times. Compute `x - 10`. How do you interpret the result?

Check other examples with different increments. In which cases can you get an exact result? Can you come up with a class of instances where the result is exact?

In [1]:
x = 0.0
for i in range(100):
    x += 0.1
print(x - 10)

# Try this instead
x = 0.0
for i in range(100):
    x += 0.25
print(x - 25)
# Why does this work?

-1.9539925233402755e-14
0.0


### 2. Computing log-probabilities

Probabilities can sometimes be difficult to compute with floating point numbers as they can be very small non-negative numbers. These problems can often be avoided by using logarithms and storing $ \log(p) $ instead of $ p $.

Compute numerically the following probabilities and report them in the format $x \cdot 10^y$:
1. The probability of randomly drawing the 8191-letter HIV-1 genome from the 4-letter DNA alphabet.
2. The probability that you need exactly 5000 throws of a regular 6-sided die to get the first 6. (*Hint*: [Geometric distribution](https://en.wikipedia.org/wiki/Geometric_distribution).)
3. The probability that $ x = 200 $ when $ x \sim \mathrm{Poisson}(1)$.

*Hints*: 
- The Python package 'numpy' contains basic numerical functions you will need. Just use `np.log()` for `log()` etc. You can use the properties of logarithms to convert natural logarithms to base 10 to make them more human-readable.
- As commonly done, in point 3 above we denoted with $x \sim P(\theta)$ that $x$ is an instance of a random variable drawn from the probability density (or probability mass function) $P$ with parameters $\theta$. In example 3, $P$ is a [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) with rate parameter $\lambda = 1$.

In [2]:
import numpy as np

# Define a function to print the values in the requested format.
# For all y, we have
#   p = 10^log10p = 10^y * 10^(log10p - y)
# where the logarithm is in base 10.
# By choosing y to be largest integer not greater than log10p, we have 1 <= x < 10.
def pretty_print_log10(log10p):
    """Print a log probability in pretty scientific notation x * 10^y."""
    y = np.floor(log10p)
    x = 10**(log10p-y)
    print("p = " + "{:.3f}".format(x) + " * 10^{:.0f}".format(y))

# Test pretty print function
pretty_print_log10(np.log10(0.00002))

#1
# Probability of drawing one letter from 4-letter alphabet is 1/4
# Assuming probabilities are independent we get Pr(genome) = 0.25^8191
logp_hiv = 8191*np.log10(0.25)
pretty_print_log10(logp_hiv)

#2
# Probability for 4999 throws before first 6 is given by geometric distribution with p = 1/6
logp_dice = 4999*np.log10(5/6)+np.log10(1/6)
pretty_print_log10(logp_dice)

#3
# Probability for x=200 when x ~ Poisson(1) is given by exp(-1)/200!.
# Logarithm of n! can be computed as the sum_i=1^n (log(i))
logp_poi = -np.log10(np.exp(1)) - sum([np.log10(i+1) for i in range(200)])
pretty_print_log10(logp_poi)

p = 2.000 * 10^-5
p = 3.362 * 10^-4932
p = 2.482 * 10^-397
p = 4.665 * 10^-376


### 3. Log-sum-exp trick when working with log-probabilities

Assuming one is working with log-probabilities as suggested above, one often runs into the need to normalise a set of log-probabilities $\textbf{x} = (x_1, \ldots, x_N)$. To do this, it is necessary to compute
$$ z = \log\left( \sum_{i=1}^N \exp(x_i) \right). $$
This can be difficult as $ \exp() $ can very easily overflow or underflow. These problems can be avoided by using the log-sum-exp (or logsumexp) trick discussed e.g. at
https://lips.cs.princeton.edu/computing-log-sum-exp/

1. Try to compute $ z $ directly for $\textbf{x} = [-1000, -999, -1000]$.
2. Compute $z$ again using the log-sum-exp trick.

In [3]:
#1
x = [-1000, -999, -1000]
z1 = np.log(np.sum(np.exp(x)))
print(z1)

#2
# log-sum-exp trick: subtract max(x) before taking exp() and add it back afterwards
def logsumexp(x):
    """Return log of sum of exponential of provided log-probability vector."""
    M = np.max(x)
    return np.log(np.sum(np.exp(x-M)))+M

z2 = logsumexp(x)
print(z2)
pretty_print_log10(z2/np.log(10))

-inf
-998.448555286068
p = 2.395 * 10^-434


C:\Users\luigi\AppData\Local\Temp/ipykernel_3192/3222279817.py:3: RuntimeWarning: divide by zero encountered in log
  z1 = np.log(np.sum(np.exp(x)))


### 4. Useful special functions

Probability distributions often involve special functions such as the [gamma function](https://en.wikipedia.org/wiki/Gamma_function) $\Gamma(z)$. The gamma function is also useful as $ n! = \Gamma(n+1) $, where $n!$ is $n$ factorial. Note that almost all numerical packages will offer a function that directly computes the *logarithm* of the Gamma function (often called something like `gammaln`).

1. Check the manual of the Python package `scipy.special` to find the different forms of gamma function it offers.
2. Redo task 3 of Exercise 2 using a suitable gamma function call from `scipy.special`.

In [4]:
#2
from scipy.special import gammaln
logp_poi = (-1 - gammaln(201))*np.log10(np.exp(1))
pretty_print_log10(logp_poi)

p = 4.665 * 10^-376


### 5. Numerical algorithms

As an example of a numerical computation, let us consider the estimation of the variance of $ n $ numbers $ x_1, \dots, x_n $.

Denoting the mean of the numbers by $ \bar{x}, $ the unbiased estimate of the sample variance is
$$ s^2 = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})^2 =
  \frac{1}{n-1} \sum_{i=1}^n (x_i^2 - 2 x_i \bar{x} + \bar{x}^2) =
  \frac{1}{n-1} \left(\sum_{i=1}^n x_i^2 - 2 n \bar{x}^2 + n \bar{x}^2\right) =
  \frac{1}{n-1} \left(\sum_{i=1}^n x_i^2 - n \bar{x}^2\right) =
  \frac{1}{n-1} \left(\sum_{i=1}^n x_i^2 - \frac{1}{n} (\sum_{i=1}^n x_i)^2\right).
$$

The variance can be estimated in a numerically stable manner using the first form, but this requires two passes through the data: first to compute the mean and then the second time to compute the sum of squared differences. The last form can be evaluated in single pass, but computing the difference of two potentially large positive numbers is numerically unstable.

1. Write a function for computing the variance of a given array of numbers using the two-pass approach:
$$ \bar{x} = \frac{1}{n} \sum_{i=1}^n x_i $$
$$ s^2 = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})^2 $$
2. Write a function for computing the variance of a given array of numbers using the one-pass approach:
$$ s^2 = \frac{1}{n-1} \left(\sum_{i=1}^n x_i^2 - \frac{1}{n} (\sum_{i=1}^n x_i)^2\right). $$
3. Test your functions by generating 1000 random number from the distribution $ N(10^9, 1) $. (*Hint*: `numpy.random.randn()`)
4. Implement Welford's accurate one-pass algorithm and test it with your data. (See e.g. http://jonisalonen.com/2013/deriving-welfords-method-for-computing-variance/)

In [5]:
import numpy as np
import numpy.random as npr

#1
# Slow two-pass implementation with loops
def two_pass_loop(x):
    mean = 0
    n = len(x)
    for x_i in x:
        mean += x
    mean /= n
    variance = 0
    for x_i in x:
        variance += (x_i - mean)**2
    variance /= n-1
    return variance

# Faster two-pass implementation using NumPy functions
def two_pass(x):
    n = len(x)
    mean = np.mean(x)
    variance = 1/(n-1)*np.sum((x-mean)**2)
    return variance

#2
# Slow one-pass implementation with a loop
def one_pass_loop(x):
    n = len(x)
    square_sum_x = 0
    sum_x = 0
    for x_i in x:
        square_sum_x += x_i**2
        sum_x += x_i
    return (square_sum_x-sum_x**2/n)/(n-1)

#3
sample = npr.normal(1e9, 1, size=1000)
print(two_pass(sample))  # variance of sample computed using two-pass approach
print(one_pass_loop(sample))  # variance of sample computed using one-pass approach

#4
# NOTE: This pure Python implementation is inefficient - you should always use NumPy functions instead!
def welfords(x):
    m = 0
    s = 0
    for k, x_i in enumerate(x):
        oldm = m
        m += (x_i-m)/(k+1)
        s+= (x_i-m)*(x_i-oldm)
    return s/k # note that indexing of array in python starts from 0 and ends to length(array)-1
                 # so at the end of for loop k=len(x)-1

print(welfords(sample))

1.0773882831126873
1443.2352352352352
1.0773882503378813


### Bonus extra: Early history of digital computers

Statistics and computers have a long common history, and the first electronic computer Colossus was built by the British to perform statistical computations for breaking a German cryptosystem during World War II. This relatively unknown part of history is reported in detail in
http://www.rutherfordjournal.org/article030109.html